# Scratchpad

A notebook for quickly iterating ideas


In [1]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
from src.crossword.utils import load_puzzle

# Load environment variables from .env file
load_dotenv()


True

In [2]:
# Load the puzzle
puzzle = load_puzzle("data/easy.json")

In [3]:
print('--- Clues ---')
clue = puzzle.clues[0]
print(clue)

--- Clues ---
number=1 text='Feline friend' direction=<Direction.ACROSS: 'across'> length=3 row=0 col=0 answer='CAT' answered=False


In [4]:
print('--- Set a guess ---')
puzzle.set_clue_chars(puzzle.clues[0], ["a", "b", "c"])
print(puzzle)

--- Set a guess ---
┌───────────────┐
│ A  B  C  ░  ░ │
│    ░     ░  ░ │
│    ░     ░  ░ │
│ ░  ░     ░  ░ │
│ ░  ░  ░  ░  ░ │
└───────────────┘


In [5]:
print('--- Undo ---')
puzzle.undo()
print(puzzle)

--- Undo ---
┌───────────────┐
│          ░  ░ │
│    ░     ░  ░ │
│    ░     ░  ░ │
│ ░  ░     ░  ░ │
│ ░  ░  ░  ░  ░ │
└───────────────┘


In [6]:
print('--- Set a guess for clue 1 ---')
puzzle.set_clue_chars(puzzle.clues[0], ["c", "a", "t"])

print('--- Set a guess for clue 2 ---')
puzzle.set_clue_chars(puzzle.clues[1], ["c", "o", "w"])

print('--- Completed all? ---')
print(puzzle.validate_all())
print(puzzle)

--- Set a guess for clue 1 ---
--- Set a guess for clue 2 ---
--- Completed all? ---
False
┌───────────────┐
│ C  A  T  ░  ░ │
│ O  ░     ░  ░ │
│ W  ░     ░  ░ │
│ ░  ░     ░  ░ │
│ ░  ░  ░  ░  ░ │
└───────────────┘


In [7]:
print('--- Set a guess for clue 3 ---')
puzzle.set_clue_chars(puzzle.clues[2], ["t", "e", "a","r"])

print('--- Completed all? ---')
print(puzzle.validate_all())
print(puzzle)

--- Set a guess for clue 3 ---
--- Completed all? ---
True
┌───────────────┐
│ C  A  T  ░  ░ │
│ O  ░  E  ░  ░ │
│ W  ░  A  ░  ░ │
│ ░  ░  R  ░  ░ │
│ ░  ░  ░  ░  ░ │
└───────────────┘


In [8]:
print('--- Reset ---')
puzzle.reset()
print(puzzle)

--- Reset ---
┌───────────────┐
│          ░  ░ │
│    ░     ░  ░ │
│    ░     ░  ░ │
│ ░  ░     ░  ░ │
│ ░  ░  ░  ░  ░ │
└───────────────┘


In [9]:
print('--- OpenAI Hello World ---')
def openai_hello_world():
    client = AzureOpenAI(
        api_version=os.getenv("OPENAI_API_VERSION"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("OPENAI_API_KEY")
    )
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a helpful assistant."}, 
                  {"role": "user", "content": "Hello!"}]
    )
    return response.choices[0].message.content

client = AzureOpenAI(
        api_version=os.getenv("OPENAI_API_VERSION"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("OPENAI_API_KEY")
    )

def query(query : str, system_prompt: str = "You are a helpful assistant.") -> str:

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": system_prompt}, 
                  {"role": "user", "content": query}]
    )
    return response.choices[0].message.content
    

print(openai_hello_world())

--- OpenAI Hello World ---
Hi there! How can I assist you today?


In [ ]:
import os
from typing import List, Dict, Optional

import logging

from src.crossword.utils import load_puzzle, CrosswordPuzzle
from src.crossword.types import Clue

logging.basicConfig(level=logging.INFO)


class Solver:
    def __init__(self, puzzle: CrosswordPuzzle):
        """
        Initialize the solver with a CrosswordPuzzle instance.
        
        Args:
            puzzle (CrosswordPuzzle): The crossword puzzle to solve.
        """
        self.puzzle = puzzle
        self.client = AzureOpenAI(
            api_version=os.getenv("OPENAI_API_VERSION"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("OPENAI_API_KEY")
        )

    def make_guess(self, clue: Clue) -> Optional[str]:
        """
        Use the OpenAI API to make a guess for the given clue.
        
        Args:
            clue (Clue): The clue for which to generate a guess.
        
        Returns:
            Optional[str]: The guessed answer, or None if no answer is generated.
        """

        logging.info(f"Making a guess for clue: {clue}")
        prompt = self._generate_prompt(clue)

        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an expert crossword-solving assistant and winner of all major cross-word competitions."},
                {"role": "user", "content": prompt}
            ]
        )

        guess = response.choices[0].message.content.strip().lower()

        logging.info(f"Guess: {guess}")
        
        return guess if len(guess) == clue.length else None

    def solve(self):
        """
        Attempt to solve the entire crossword by making guesses for each clue.
        """
        for clue in self.puzzle.clues:
            logging.info(f"Solving clue: {clue}")
            if not clue.answered:
                guess = self.make_guess(clue)

                if guess:
                    self.puzzle.set_clue_chars(clue, list(guess))

    def validate_solution(self) -> bool:
        """
        Validate the solution of the crossword.
        
        Returns:
            bool: True if all clues are correctly solved, otherwise False.
        """
        validation = self.puzzle.validate_all()

        if validation:
            logging.info("Puzzle solved!")
        
        else:
            logging.info("Puzzle not solved.")

        return validation

    @staticmethod
    def _generate_prompt(clue: Clue) -> str:
        """
        Generate a suitable prompt for the OpenAI API to guess the clue answer.
        
        Args:
            clue (Clue): The clue for which to generate a prompt.
        
        Returns:
            str: The formatted prompt.
        """
        return (
            f"Please solve the following crossword clue:\n\n"
            f"Clue: {clue.text}\n"
            f"Direction: {'Across' if clue.direction == 'across' else 'Down'}\n"
            f"Length: {clue.length}\n"
            #f"Known Letters: {''.join(ch or '_' for ch in clue.cells())}\n\n"
            f"Provide your best guess as a single word in lowercase, "
            f"exactly {clue.length} characters long."
            "You must return ONLY the answer to the clue, not the clue itself and not any other text."
        )


In [29]:
puzzle = load_puzzle("data/easy.json")

solver = Solver(puzzle)

solver.solve()

solver.validate_solution()


INFO:root:Solving clue: number=1 text='Feline friend' direction=<Direction.ACROSS: 'across'> length=3 row=0 col=0 answer='CAT' answered=False
INFO:root:Making a guess for clue: number=1 text='Feline friend' direction=<Direction.ACROSS: 'across'> length=3 row=0 col=0 answer='CAT' answered=False
INFO:httpx:HTTP Request: POST https://i-dot-ai-interview.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-10-01-preview "HTTP/1.1 200 OK"
INFO:root:Guess: cat
INFO:root:Solving clue: number=1 text='Dairy farm animal' direction=<Direction.DOWN: 'down'> length=3 row=0 col=0 answer='COW' answered=False
INFO:root:Making a guess for clue: number=1 text='Dairy farm animal' direction=<Direction.DOWN: 'down'> length=3 row=0 col=0 answer='COW' answered=False
INFO:httpx:HTTP Request: POST https://i-dot-ai-interview.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-10-01-preview "HTTP/1.1 200 OK"
INFO:root:Guess: cow
INFO:root:Solving clue: number=2 t

True

In [33]:
from typing import List
import logging

def get_ranked_answers(clue: Clue, client, attempt = 0, n_answers = 5) -> List[str]:
    """
    Use the OpenAI API to get three possible answers for a clue ranked in confidence order.
    
    Args:
        clue (Clue): The clue for which to generate answers.
        client: The OpenAI client instance.
    
    Returns:
        List[str]: A list of three ranked answers for the clue.
    """
    prompt = (
        f"Please solve the following crossword clue:\n\n"
        f"Clue: {clue.text}\n"
        f"Direction: {'Across' if clue.direction == 'across' else 'Down'}\n"
        f"Length: {clue.length}\n"
        f"Provide your top {n_answers} answers as a single word for each, "
        f"ranked in order of confidence from highest to lowest, "
        f"separated by commas. Return only the answers."
    )

    logging.info(f"Generating ranked answers for clue: {clue.text}")

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an expert crossword-solving assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    try:
        answers = response.choices[0].message.content.strip().lower().split(",")

    except AttributeError as e:
        logging.error(f"Error generating answers: {e}")
        

    ranked_answers = [answer.strip() for answer in answers]

    # Filter out any answers that are not the correct length.
    ranked_answers = [answer for answer in ranked_answers if len(answer) == clue.length]

    logging.info(f"Ranked answers: {ranked_answers}")

    if (not ranked_answers) and attempt < n_answers:
        logging.info("No valid answers generated. Retrying.")
        return get_ranked_answers(clue, client, attempt + 1)

    return ranked_answers[:n_answers]  # Ensure only the top three are returned.


In [34]:
from itertools import product
from typing import List
from src.crossword.exceptions import InvalidClueError

class CombinationSolver:
    def __init__(self, puzzle: CrosswordPuzzle, client):
        """
        Initialize the combination solver with a crossword puzzle and OpenAI client.
        
        Args:
            puzzle (CrosswordPuzzle): The crossword puzzle to solve.
            client: The OpenAI client for generating answers.
        """
        self.puzzle = puzzle
        self.client = client
        self.clue_answers = {}

    def generate_clue_answers(self):
        """
        Generate three ranked answers for each clue in the puzzle.
        """
        for clue in self.puzzle.clues:
            if not clue.answered:
                self.clue_answers[clue] = get_ranked_answers(clue, self.client)

    def solve_with_combinations(self):
        """
        Try all combinations of ranked answers for the clues and validate the solution.
        
        Returns:
            bool: True if a valid solution is found, otherwise False.
        """
        self.generate_clue_answers()

        logging.info("Clue answers:")
        for clue, answers in self.clue_answers.items():
            logging.info(f"Clue {clue}: {answers}")

        # Generate all combinations of ranked answers
        clue_combinations = product(
            *[self.clue_answers[clue] for clue in self.puzzle.clues if not clue.answered]
        )

        for combination in clue_combinations:
            logging.info(f"Trying combination: {combination}")
            self.puzzle.reset()

            try:

                # Apply each answer to the corresponding clue
                for clue, answer in zip(self.puzzle.clues, combination):
                    if not clue.answered:
                        self.puzzle.set_clue_chars(clue, list(answer))

            except InvalidClueError as e:
                logging.info(f"Invalid clue encountered: {e}")
                continue

            # Validate the solution
            if self.puzzle.validate_all():
                logging.info("Valid solution found!")
                return True

        logging.info("No valid solution found.")
        return False



In [32]:
puzzle = load_puzzle("data/medium.json")

solver = CombinationSolver(puzzle, client)

solver.solve_with_combinations()



INFO:root:Generating ranked answers for clue: A long narrative poem
INFO:httpx:HTTP Request: POST https://i-dot-ai-interview.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-10-01-preview "HTTP/1.1 200 OK"
INFO:root:Ranked answers: ['epic', 'saga', 'epoe', 'song', 'ball']
INFO:root:Generating ranked answers for clue: A person who writes books
INFO:httpx:HTTP Request: POST https://i-dot-ai-interview.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-10-01-preview "HTTP/1.1 200 OK"
INFO:root:Ranked answers: ['author', 'writer']
INFO:root:Generating ranked answers for clue: A short story with a moral lesson
INFO:httpx:HTTP Request: POST https://i-dot-ai-interview.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-10-01-preview "HTTP/1.1 200 OK"
INFO:root:Ranked answers: ['parable']
INFO:root:Generating ranked answers for clue: A book's outer casing
INFO:httpx:HTTP Request: POST https://i-dot-ai-interview.op

True

In [35]:
puzzle = load_puzzle("data/hard.json")

solver = CombinationSolver(puzzle, client)

solver.solve_with_combinations()



INFO:root:Generating ranked answers for clue: Greek tragedy (7,3)


INFO:httpx:HTTP Request: POST https://i-dot-ai-interview.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-10-01-preview "HTTP/1.1 200 OK"
INFO:root:Ranked answers: []
INFO:root:No valid answers generated. Retrying.
INFO:root:Generating ranked answers for clue: Greek tragedy (7,3)
INFO:httpx:HTTP Request: POST https://i-dot-ai-interview.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-10-01-preview "HTTP/1.1 200 OK"
INFO:root:Ranked answers: []
INFO:root:No valid answers generated. Retrying.
INFO:root:Generating ranked answers for clue: Greek tragedy (7,3)
INFO:httpx:HTTP Request: POST https://i-dot-ai-interview.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-10-01-preview "HTTP/1.1 200 OK"
INFO:root:Ranked answers: []
INFO:root:No valid answers generated. Retrying.
INFO:root:Generating ranked answers for clue: Greek tragedy (7,3)
INFO:httpx:HTTP Request: POST https://i-dot-ai-interview.openai.azure.

KeyboardInterrupt: 